In [27]:
pip install langchain langchain-community langchain-google-genai faiss-cpu


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
# Load your text file
from langchain.document_loaders import TextLoader
loader = TextLoader("data/myfile.txt")
documents = loader.load()

In [29]:
# Split the documents into chunks
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(separator="\n", chunk_size=200, chunk_overlap=20)
split_docs = splitter.split_documents(documents)

print(f"Loaded {len(documents)} documents")
print(f"Split into {len(split_docs)} chunks")
if len(split_docs) == 0:
    print("No chunks to embed! Check your file and splitter settings.")


Created a chunk of size 576, which is longer than the specified 200
Created a chunk of size 768, which is longer than the specified 200
Created a chunk of size 404, which is longer than the specified 200
Created a chunk of size 307, which is longer than the specified 200
Created a chunk of size 324, which is longer than the specified 200
Created a chunk of size 510, which is longer than the specified 200
Created a chunk of size 317, which is longer than the specified 200
Created a chunk of size 505, which is longer than the specified 200
Created a chunk of size 488, which is longer than the specified 200
Created a chunk of size 437, which is longer than the specified 200
Created a chunk of size 272, which is longer than the specified 200
Created a chunk of size 330, which is longer than the specified 200


Loaded 1 documents
Split into 26 chunks


In [ ]:
# Set your Gemini API key
import os
os.environ["GOOGLE_API_KEY"] = "your-gemini-api-key"


In [31]:
# Use Gemini embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(split_docs, embeddings)

In [32]:
#get from user
prompt = input()

In [33]:
results = db.similarity_search(prompt, k=3)

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
prompt_template = ChatPromptTemplate.from_template(
    "Answer the question based only on the following documents:\n{context}\n\nQuestion: {question}"
)
qa_chain = create_stuff_documents_chain(llm, prompt_template)

response = qa_chain.invoke({"context": results, "question": prompt})
print(response)

According to the provided documents, IIT Kanpur mentored IIT Jodhpur.
